# SPARK ML (Machine Learning with Spark)

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=782360d7bb39368b483681c9e1321c4b455124cb750bc492193e0a7f39932b96
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Creating Session and Context

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [ ]:
libsvm = '''
1 1:3.0 4:5.0 20:1.0
0 1:3.0 5:5.0 12:1.0
1 2:1.0 3:2.0 15:1.0
'''

open("sample_libsvm.txt","w").write(libsvm)

df_toy = spark.read.format('libsvm').load('./sample_libsvm.txt')

df_toy.show()


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(20,[0,3,19],[3.0...|
|  0.0|(20,[0,4,11],[3.0...|
|  1.0|(20,[1,2,14],[1.0...|
+-----+--------------------+



### Creating a Data Frame as running example

In [ ]:
data = [("Joe","The movie was disapointing", 0),
        ("Mae","I liked very much the film", 1),
        ("Joe","What ugly end, didn't like at all ", 0),
        ("Mick","This film makes me happy", 1),
        ("Mae","Fantastic argument!",1)]

columns = ["reviewer","text","label"]

df = spark.createDataFrame(data,columns)

df.printSchema()
df.show()

root
 |-- reviewer: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: long (nullable = true)

+--------+--------------------+-----+
|reviewer|                text|label|
+--------+--------------------+-----+
|     Joe|The movie was dis...|    0|
|     Mae|I liked very much...|    1|
|     Joe|What ugly end, di...|    0|
|    Mick|This film makes m...|    1|
|     Mae| Fantastic argument!|    1|
+--------+--------------------+-----+



Splitting the data into train/test partitions:

In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
train.count()

4

### Mounting the Pipeline, learn and predict:

In [ ]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20) #for real text should be 1000-2000

idf = IDF(inputCol="rawFeatures", outputCol="features")

nb = NaiveBayes(modelType="multinomial")

In [ ]:
pipeline = Pipeline(stages=[tokenizer,hashingTF,idf,nb])

In [ ]:
print(nb.explainParams())

featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default), bernoulli and gaussian. (default: multinomial, current: multinomial)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
smoothing: The smoothing parameter, should be >= 0, default is 1.0 (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest val

In [ ]:
nb.smoothing

Param(parent='NaiveBayes_8e5e6c1832a5', name='smoothing', doc='The smoothing parameter, should be >= 0, default is 1.0')

Training the model:

In [ ]:
model = pipeline.fit(train)

Predictions with the model:

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.select("label","prediction","probability").show(2)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    0|       0.0|[0.69456313923456...|
+-----+----------+--------------------+



### Evaluating the model:

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)

accuracy

1.0

We want to create a learning model for predicting the 5-star rates of the movies dataset used in the previous deliverable. Split the dataset into two partitions: train (80%) and test (20%). Choose the model you think is best suited for this task (e.g., Naive Bayes, Logistic Regression, etc.) and perform the dataset transformations you consider relevant to obtain the feature vectors for training the chosen model. Finally, evaluate the performance of the model with the appropriate evaluator over the predictions in the test dataset.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from google.colab import drive
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from xgboost.spark import SparkXGBClassifier

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Init spark session
spark = SparkSession.builder.appName("films").getOrCreate()
sc = spark.sparkContext

file_path = "/content/gdrive/MyDrive/Master_23_24/Big_Data/filmsML.txt/part-00000-145dc166-9f44-4c07-abdc-289b9a8cd6dc-c000.txt"

I have some modifications in notebook 6. I have reduced the number of features (delate stop-words, words that just appears one time and words with a tf*idf below the 25th percentile of the total words of the document).

In [ ]:
# Read the preprocesed dataset
df = spark.read.format('libsvm').load(file_path)
df.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  4.0|(40018,[0,1,2,3,5...|
|  4.0|(40018,[7,33,39,1...|
|  4.0|(40018,[43,47,133...|
|  3.0|(40018,[2,28,81,8...|
|  3.0|(40018,[2,35,43,6...|
+-----+--------------------+
only showing top 5 rows



In [ ]:
# Calcula el total de películas en el DataFrame
total_movies = df.count()

# Agrupa por la columna 'label' y cuenta las ocurrencias
count_by_label = df.groupBy('label').agg(count('*').alias('count'))

# Calcula el porcentaje sobre el total para cada clase
count_by_label_with_percentage = count_by_label.withColumn('percentage', (col('count') / total_movies) * 100)

count_by_label_with_percentage.show()

+-----+-----+------------------+
|label|count|        percentage|
+-----+-----+------------------+
|  1.0|  351| 9.051057246003094|
|  4.0|  890| 22.94997421351212|
|  3.0| 1253| 32.31046931407942|
|  2.0|  923|23.800928313563695|
|  5.0|  461|11.887570912841671|
+-----+-----+------------------+



Random Forest

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento y 20% para prueba)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Crear el clasificador RandomForest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=5)

# Construir el pipeline
pipeline = Pipeline(stages=[rf])

# Entrenar el modelo
model = pipeline.fit(train_data)

In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = model.transform(test_data)

In [ ]:
# Evaluación del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {:.2f}%".format(accuracy * 100))

Accuracy = 30.64%


In [ ]:
predictions.select("label","prediction").show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       3.0|
|  1.0|       3.0|
|  1.0|       3.0|
|  1.0|       3.0|
|  1.0|       2.0|
+-----+----------+
only showing top 5 rows



Logistic Regresion

In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Crear el clasificador RandomForest
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=3)

# Construir el pipeline
pipeline = Pipeline(stages=[lr])

# Entrenar el modelo
model = pipeline.fit(train_data)

In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = model.transform(test_data)

In [ ]:
# Evaluación del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {:.2f}%".format(accuracy * 100))

Accuracy = 38.02%


In [ ]:
predictions.select("label","prediction").show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       2.0|
|  1.0|       3.0|
|  1.0|       5.0|
|  1.0|       3.0|
|  1.0|       2.0|
+-----+----------+
only showing top 5 rows



XGBoost, I can't doing that it works

In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)

xgb = SparkXGBClassifier(features_col="features",
                         label_col="label",
                         prediction_col="prediction")


model2 = xgb.fit(train_data)

predictions2 = xgb.predict(test_data)

In [ ]:
# Evaluación del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions2)
print("Accuracy = {:.2f}%".format(accuracy * 100))

In [ ]:
predictions2.select("label","prediction").show(721)


I going to try the same algoritms but, I going to preproces the original data in a pysparkML pipeline

In [ ]:
!wget "http://krono.act.uji.es/IDIA/criticas_pelis.csv.gz"
!gunzip "criticas_pelis.csv.gz"

--2023-11-02 14:48:10--  http://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
Resolving krono.act.uji.es (krono.act.uji.es)... 150.128.97.37
Connecting to krono.act.uji.es (krono.act.uji.es)|150.128.97.37|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz [following]
--2023-11-02 14:48:10--  https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
Connecting to krono.act.uji.es (krono.act.uji.es)|150.128.97.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4447654 (4.2M) [application/x-gzip]
Saving to: ‘criticas_pelis.csv.gz.1’

criticas_pelis.csv. 100%[===================>]   4.24M  3.99MB/s    in 1.1s    

2023-11-02 14:48:12 (3.99 MB/s) - ‘criticas_pelis.csv.gz.1’ saved [4447654/4447654]

gzip: criticas_pelis.csv already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("films2").getOrCreate()

# Path to CSV file
csv = "criticas_pelis.csv"

# Load compressed CSV file into Spark DataFrame
df = spark.read.options(header=False).csv(csv)

df.show(5)

+----+---------+--------------------+---+
| _c0|      _c1|                 _c2|_c3|
+----+---------+--------------------+---+
|Row0|   File-0| May, ¿quieres se...|  4|
|Row1|   File-1| Cómo ponerse en ...|  4|
|Row2|  File-10| Deliciosa comedi...|  4|
|Row3| File-100| La ironía es el ...|  3|
|Row4|File-1000| Al final, y teni...|  3|
+----+---------+--------------------+---+
only showing top 5 rows



In [ ]:
# Cambiar el nombre de la columna "Name" a "Full Name"
# y el nombre de la columna "City" a "Location"
df = df.withColumnRenamed("_c2", "text").withColumnRenamed("_c3", "label")

# Eliminar las columnas "Age" y "Salary"
df = df.drop("_c0", "_c1")

# Mostrar el DataFrame después de eliminar las columnas
df.show(5)


+--------------------+-----+
|                text|label|
+--------------------+-----+
| May, ¿quieres se...|    4|
| Cómo ponerse en ...|    4|
| Deliciosa comedi...|    4|
| La ironía es el ...|    3|
| Al final, y teni...|    3|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Cambiar una columna de tipo string a tipo numérico
df = df.withColumn("label", col("label").cast("int"))
df.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
| May, ¿quieres se...|    4|
| Cómo ponerse en ...|    4|
| Deliciosa comedi...|    4|
| La ironía es el ...|    3|
| Al final, y teni...|    3|
+--------------------+-----+
only showing top 5 rows



In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000) #for real text should be 1000-2000

idf = IDF(inputCol="rawFeatures", outputCol="features")

rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=5)

pipeline = Pipeline(stages=[tokenizer,hashingTF,idf,rf])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento y 20% para prueba)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Entrenar el modelo
model = pipeline.fit(train_data)

In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = model.transform(test_data)

In [ ]:
# Evaluación del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {:.2f}%".format(accuracy * 100))

Accuracy = 31.81%


In [ ]:
predictions.select("label","prediction").show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|    4|       3.0|
|    3|       3.0|
|    4|       3.0|
|    2|       3.0|
|    4|       3.0|
+-----+----------+
only showing top 5 rows



Logisic Regresion

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000) #for real text should be 1000-2000

idf = IDF(inputCol="rawFeatures", outputCol="features")

lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=6)

pipeline = Pipeline(stages=[tokenizer,hashingTF,idf,lr])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento y 20% para prueba)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Entrenar el modelo
model = pipeline.fit(train_data)

In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = model.transform(test_data)

In [ ]:
# Evaluación del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {:.2f}%".format(accuracy * 100))

Accuracy = 41.25%


In [ ]:
predictions.select("label","prediction").show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|    4|       4.0|
|    3|       3.0|
|    4|       3.0|
|    2|       3.0|
|    4|       2.0|
+-----+----------+
only showing top 5 rows



After testing several algorithms, first with the preprocessed data from notebook 6 and then implementing a pipeline that takes the raw data and processes it until obtaining the appropriate data set to apply the machine learning algorithms. In both cases, Random Forest and Logistic Regression have been used, testing with different hyperparameters, a maximum precision of 41.25% has been obtained using Logistic Regression. This very low performance of the model is mainly due to the fact that we are using as characteristics the TF*IDF of the words that form the reviews of the movies, this being a not very powerful metric to perform tasks of this type, compared to current techniques of natural language processing that uses embeddings (dense vectors in which the text is encoded) that are capable of capturing the semantics of different text sequences, which allows obtaining a richer and more meaningful representation of the textual content. Embeddings are capable of capturing semantic and contextual relationships between words, resulting in better understanding of the data by the machine learning model.

In contrast, TF*IDF simply assigns a weight to each word based on its frequency in a document and its inverse frequency in the data set. Although this technique can be useful in certain contexts, such as information retrieval or document classification, it is not sufficient to capture the semantic complexity of movie reviews.

To improve model performance, one could consider adopting more advanced natural language processing techniques, such as using pre-trained language models such as BERT, GPT, or FastText. These models are capable of generating contextualized embeddings, which means that the representation of a word can vary depending on the context in which it appears in the sentence. This provides a deeper understanding of the structure and meaning of the text.